In [3]:
import sys
import os

# Add project root to sys.path
notebook_dir = os.getcwd() 
project_root = os.path.abspath(os.path.join(notebook_dir, '..')) 
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.models.v2_pool import UniswapV2Pool
from src.models.v4_gamma_pool import v4_gamma_hook_pool
from src.models.buy_and_hold import BuyAndHold
from src.utils.clean import sweep_swap_amount_range, get_symbol, get_xy_fig, get_bh_df
import plotly.express as px
import pandas as pd
import numpy as np  
import plotly.graph_objects as go


In [ ]:
parameters = {
    'fee_pct': 0.003, # Example: 30% fee for clear effect in plot ONLY AFFECT V2POOL with FEE   
    'initial_x': float(100), # 1 hunded USDC
    'initial_y': float(1),   # 1 ARB
    'num_points': 1000, # Number of relative amounts to test
    'max_relative_amount': 2, # Test swaps with amount_in up to 5x of reserve
    'gamma_factor': 0.3 # 30% gamma factor for clear effect in plot ONLY AFFECT V4 GAMMA POOL with GEE
}

v2_pool_0_fee = UniswapV2Pool(parameters, is_zero_fee=True)
v2_swap_sweep_results_0_fee_df = sweep_swap_amount_range(v2_pool_0_fee,parameters)
v2_swap_sweep_results_0_fee_df['O_R']=0

# Calculate 'O_R' using vectorized operations with np.where
condition = v2_swap_sweep_results_0_fee_df['token_in_is_USDC'] == True
value_if_true = v2_swap_sweep_results_0_fee_df['amount_out'] / v2_swap_sweep_results_0_fee_df['new_y']
value_if_false = v2_swap_sweep_results_0_fee_df['amount_in'] / v2_swap_sweep_results_0_fee_df['new_y']

v2_swap_sweep_results_0_fee_df['O_R'] = np.where(condition, value_if_true, value_if_false)


fig = get_xy_fig(v2_swap_sweep_results_0_fee_df,'O_R','IL_O', 'OrderSize to Reseve vs IL to OrderSize','Order pct of Reserve','IL pct of Order')
fig.update_layout(
    xaxis_range=[0, 1],
    yaxis_range=[0, 1]
)
fig.show()

In [3]:



v2_pool_0_fee = UniswapV2Pool(parameters, is_zero_fee=True)
v2_swap_sweep_results_0_fee_df = sweep_swap_amount_range(v2_pool_0_fee,parameters)

v2_pool_flat_fee = UniswapV2Pool(parameters, is_zero_fee=False)
v2_swap_sweep_results_f_fee_df = sweep_swap_amount_range(v2_pool_flat_fee,parameters)

v4_gamma_hook_pool_0_gamma = v4_gamma_hook_pool(parameters, is_zero_gee=True)
v4_gamma_hook_sweep_results_0_gee_gamma_df = sweep_swap_amount_range(v4_gamma_hook_pool_0_gamma,parameters)

v4_gamma_hook_pool_gamma = v4_gamma_hook_pool(parameters, is_zero_gee=False)
v4_gamma_hook_sweep_results_g_gee_gamma_df = sweep_swap_amount_range(v4_gamma_hook_pool_gamma,parameters)

buy_and_hold = BuyAndHold(parameters)
buy_and_hold_df = get_bh_df(buy_and_hold, v2_swap_sweep_results_0_fee_df)



joint_df = pd.concat([
    v2_swap_sweep_results_0_fee_df, 
    v2_swap_sweep_results_f_fee_df, 
    v4_gamma_hook_sweep_results_0_gee_gamma_df, 
    v4_gamma_hook_sweep_results_g_gee_gamma_df, 
    buy_and_hold_df])


In [ ]:
fig = get_xy_fig(joint_df, 'new_price', 'lp_value', 'LP Value vs. New Price (1 swap)', 'ARB Price ($)', 'Holder Value ($)')
fig.update_layout(
    xaxis_range=[0, 300],
    yaxis_range=[0, 400]
)

fig.show()

In [ ]:
# this plot requires for very extreme max_relative amount to reveal iteself fully - please adjust parameters and increase max_relative amount and num_points to see the full plot. Leaving defaults 
fig = get_xy_fig(joint_df, 'new_x', 'new_y', 'XY for 1 swap', 'new_x', 'new_y')    

fig.update_layout(
    yaxis_range=[0, 100],
    xaxis_range=[0, 100]
)

fig.show()

### Now we look at how the Gamma of each scenario behaves


In [ ]:
fig = get_xy_fig(joint_df, 'new_price', 'gamma', 'Gamma vs. Price', 'ARB Price ($)', 'Gamma')

fig.update_layout(
    yaxis_range=[-0.001, 0.001]
)
fig.show()






In [ ]:
fig = get_xy_fig(joint_df, 'new_price', 'delta', 'Delta vs. New Price', 'ARB Price ($)', 'delta')

fig.update_layout(
    yaxis_range=[-1, 3],
    xaxis_range=[0, 200]
)

fig.show()

In [ ]:
fig = get_xy_fig(joint_df, 'order_size_usdc', 'swap_price', 'Order Size vs Price', 'Order Size (USDC)', 'Price ARB ($)')

fig.update_layout(
    yaxis_range=[0, 300],
    xaxis_range=[-100, 100]
)

fig.show()

In [ ]:
fig = get_xy_fig(joint_df, 'order_size_usdc', 'impermanent_loss_usdc', 'Order Size vs Impermanent Loss', 'Order Size (USDC)', 'Impermanent Loss ($)')

fig.update_layout(
    yaxis_range=[-100, 100]
)

fig.show()

In [ ]:
fig = get_xy_fig(joint_df, 'new_price', 'impermanent_loss_usdc', 'Price vs Impermanent Loss', 'Price ARB ($)', 'Impermanent Loss ($)')

fig.update_layout(
    yaxis_range=[-100, 100],
    xaxis_range=[0, 200]
)

fig.show()

In [ ]:
fig = get_xy_fig(joint_df, 'relative_amount_in', 'impermanent_loss_usdc', 'Relative Amount vs Impermanent Loss', 'Relative Amount (Order Size / Reserve)', 'Impermanent Loss ($)')

fig.update_layout(
    yaxis_range=[-100, 100]
)

fig.show()

In [ ]:
fig = get_xy_fig(joint_df, 'relative_amount_in', 'total_fee_pct', 'Fee pct on incoming order', 'Relative Amount (Order Size / Reserve)', 'Fee pct')

fig.update_layout(
    xaxis_range=[-1, 1],
    yaxis_range=[-1, 1]
)

fig.show()

### SIMULATION TRADES


In [4]:
from utils.clean import generate_arb_trades, generate_usdc_trades, simulate_trade_sequence, get_3_plots, get_xy_fig
from models.v2_pool import UniswapV2Pool
from models.v4_gamma_pool import v4_gamma_hook_pool
from utils.clean import run_simulation 
import pandas as pd

parameters = {
    'num_trades_to_simulate': 15000, # Or use the desired number
    'fee_pct': 0.003, # Using 30% fee for clear effect in plot ONLY AFFECT V2POOL with FEE
    'initial_x': 100,  # 100 USDC
    'initial_y': 100,   # 100 ARB (as starting point, 1 ARB worth 1 USDC for easy math and same scale visuals)
    'gamma_factor': 0.3 # 30% gamma factor for clear effect in plot ONLY AFFECT V4 GAMMA POOL with GEE
}

arb_samples = generate_arb_trades(parameters['num_trades_to_simulate'])
arb_samples = arb_samples[arb_samples > 0]
usdc_samples = generate_usdc_trades(parameters['num_trades_to_simulate'])
usdc_samples = usdc_samples[usdc_samples > 0]

num_plot_trades = int(parameters['num_trades_to_simulate']*0.45) # Keep this smaller for faster initial testing/plotting if needed - also only positives showing
arb_samples_run = arb_samples[:num_plot_trades]
usdc_samples_run = usdc_samples[:num_plot_trades]


Reading data from: /Users/jordi/Documents/GitHub/v4-gamma-hook/data/swap_fee_data.csv
Reading data from: /Users/jordi/Documents/GitHub/v4-gamma-hook/data/swap_fee_data.csv


In [13]:
usdc_repeats=[0.25,0.25,0.25,0.25]
# We start with a zero fee pool
pool = UniswapV2Pool(parameters, is_zero_fee=True)

trade_simulation_0_fee = simulate_trade_sequence(
    pool,
    usdc_repeats,
    usdc_repeats,
    parameters
)

fig = get_xy_fig(trade_simulation_0_fee, 'trade_number', 'amount_out', 'Amount out vs Trade Number', 'Trade Number', 'Amount out')
fig.show()


In [26]:
usdc_repeats=[0.25,0.25,0.25,0.25]
# We start with a zero fee pool 
pool = v4_gamma_hook_pool(parameters, is_zero_gee=True)

trade_simulation_0_fee = simulate_trade_sequence(
    pool,
    usdc_repeats,
    usdc_repeats,
    parameters
)

fig = get_xy_fig(trade_simulation_0_fee, 'trade_number', 'amount_out', 'Amount out vs Trade Number', 'Trade Number', 'Amount out')
fig.show()

In [ ]:

# We start with a zero fee pool
pool = UniswapV2Pool(parameters, is_zero_fee=True)

trade_simulation_0_fee = simulate_trade_sequence(
    pool,
    arb_samples_run,
    usdc_samples_run,
    parameters
)

fig_combined = get_3_plots(trade_simulation_0_fee, f'Trade simulation:V2 Pool with 0% fee')
fig_combined.show()


In [ ]:

pool = UniswapV2Pool(parameters, is_zero_fee=False)

trade_simulation_f_fee = simulate_trade_sequence(
    pool,
    arb_samples_run,
    usdc_samples_run,
    parameters
)

fig_combined = get_3_plots(trade_simulation_f_fee, f'Uni V2 with {parameters["fee_pct"]*100}% fee')
fig_combined.show()


In [ ]:

pool = v4_gamma_hook_pool(parameters, is_zero_gee=True)

trade_simulation_v4_0_gee = simulate_trade_sequence(
    pool,
    arb_samples_run,
    usdc_samples_run,
    parameters
)

fig_combined = get_3_plots(trade_simulation_v4_0_gee, f'V4 Gamma Hook with 0% g')
fig_combined.show()


In [ ]:

pool = v4_gamma_hook_pool(parameters, is_zero_gee=False)

trade_simulation_v4_g_gee = simulate_trade_sequence(
    pool,
    arb_samples_run,
    usdc_samples_run,
    parameters
)

fig_combined = get_3_plots(trade_simulation_v4_g_gee, f'V4 Gamma Hook with {parameters["gamma_factor"]*100}% Gee')
fig_combined.show()


In [ ]:
joint_df = pd.concat([
    trade_simulation_0_fee, 
    trade_simulation_f_fee, 
    trade_simulation_v4_0_gee, 
    trade_simulation_v4_g_gee, 
    ])


fig = get_xy_fig(joint_df, 'trade_number', 'price', 'Resulting Swap Prices (identical relative order sizes)', 'Trade Number', 'Price ARB ($)')
fig.show()

In [ ]:
fig = get_xy_fig(joint_df, 'trade_number', 'impermanent_loss', 'Trade Number vs Impermanent Loss', 'Trade Number', 'Impermanent Loss ($)')
fig.show()

In [ ]:
from utils.clean import run_simulation # This is the function that runs the simulation

parameters = {
        'num_trades_to_simulate': 20000, # Or use the desired number
        'fee_pct': 0.003, # Using 30% fee for clear effect in plot ONLY AFFECT V2POOL with FEE
        'initial_x': 100,  # 100 USDC
        'initial_y': 100,   # 100 ARB (as starting point, 1 ARB worth 1 USDC for easy math and same scale visuals)
        'gamma_factor': 0.3 # 30% gamma factor for clear effect in plot ONLY AFFECT V4 GAMMA POOL with GEE
    }

joint_df = run_simulation(parameters)

fig = get_xy_fig(joint_df, 'trade_number', 'impermanent_loss', 'Trade Number vs Impermanent Loss', 'Trade Number', 'Impermanent Loss ($)')
fig.show()

In [ ]:

fig = get_xy_fig(joint_df, 'fee_pct', 'impermanent_loss_pct', f'Fee pct vs Impermanent Loss ({parameters["num_trades_to_simulate"]} trades)', 'Fee pct', 'Impermanent Loss (%)')
for trace in fig.data:
    if hasattr(trace, 'marker') and trace.mode == 'markers':
        trace.marker.opacity = 0.1 # Set desired transparency (e.g., 0.5 for 50%)

fig.show()

In [ ]:

# --- Simulation Parameters --- 
parameters = {
    'num_trades_to_simulate': 10000, # Trades per simulation cycle
    'fee_pct': 0.003,
    'initial_x': 100,
    'initial_y': 100,
    'gamma_factor': 0.3
}

# --- Monte Carlo Setup ---
num_runs = 100  # Number of simulation cycles to run EXPENSIVE! WARNING - MAY TAKE A WHILE TO RUN IF YOU SET THIS HIGH
# monte_carlo_results_df = pd.DataFrame()
monte_carlo_summary_list = []

# --- Run Monte Carlo Simulation ---
print(f"Starting Monte Carlo simulation for {num_runs} runs...")
# Using tqdm for progress bar
for i in range(num_runs):
    # Run one simulation cycle and get the final states
    final_states_df = run_simulation(parameters)
    final_states_df['run_id'] = i

    unique_pool_names = final_states_df['pool_name'].unique()

    for pool_name in unique_pool_names:
        pool_name_df = final_states_df[final_states_df['pool_name'] == pool_name]
        total_fee_pct = pool_name_df['fee_pct'].sum()
        average_fee_pct = total_fee_pct / len(pool_name_df)
        median_fee_pct = pool_name_df['fee_pct'].median()

        final_states_new_row = pool_name_df.iloc[-1].copy()
        final_states_new_row['total_fee_pct'] = total_fee_pct
        final_states_new_row['average_fee_pct'] = average_fee_pct   
        final_states_new_row['median_fee_pct'] = median_fee_pct

        # Append results if the simulation was successful
        monte_carlo_summary_list.append(final_states_new_row)



print("Monte Carlo simulation finished.")

monte_carlo_results_df = pd.DataFrame(monte_carlo_summary_list)



In [ ]:

pool_count = monte_carlo_results_df['pool_name'].nunique()
num_runs = int(monte_carlo_results_df.shape[0] / pool_count)
trade_count = max(monte_carlo_results_df['trade_number'])+1
fig = get_xy_fig(monte_carlo_results_df, 'average_fee_pct', 'impermanent_loss_pct', f'Monte Carlo Simulation Results - {num_runs} runs - each run {trade_count} trades', 'Average Fee pct', 'Impermanent Loss (%)')

fig.show()